In [1]:
## get data
import wrangle

## analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

**Formula**  

- for each states:
    - **Note:** proportion of ppl get sick who are in labor force (current unemployment rate, by age)

$$unquarantine\_ change\_ rate=\sum_{i=0}^{n} \sum_{j=0}^{m}{proportion\_of\_state\_sick_{i,j}}\times{proportion\_of\_inds\_in\_state_{i,j}}$$
                        - n: number of states 
                        - m: number of industries


- Depending on how much data we have, we could put _shock_ in the formula (financial crsis, hospital bed overflow) 
    
- quarantine_change_rate (come back to this)
    - we should not assume quarantine rate for all the states, might need diff quarantine states
    - Jenn: current quarantine measures for each states
    - WFH possibility will diff from industry
    - layoffs

- Health care cost 

## Data
- employment 2019
- covid 19 confirmed cases
- states populations :https://www.census.gov/quickfacts/NY


In [2]:
employment_ny = wrangle.get_ny_employment()

In [17]:
employment_ny.head()

,naics_code,industry,2019,2021,net_change,pct_change
1,601,"Total Self Employed, All Jobs",570590,577460,6870,0.0120402
2,11,"Agriculture, Forestry, Fishing and Hunting",31180,31870,690,0.0221296
8,21,Mining,4230,4550,320,0.0756501
12,22,Utilities,36700,36430,-270,-0.00735695
13,23,Construction,380940,390570,9630,0.0252796


In [3]:
covid19_us = wrangle.get_covid_data(date = "03-29-2020")

In [10]:
ny_confirmed = covid19_us.loc[covid19_us["Province_State"] == "New York", "Confirmed"].sum()

In [5]:
population_2019 = wrangle.get_population_data()

In [6]:
population_2019.head()

,SUMLEV,REGION,DIVISION,STATE,NAME,POPESTIMATE2019,POPEST18PLUS2019,PCNT_POPEST18PLUS
0,10,0,0,0,United States,328239523,255200373,77.7
1,40,3,6,1,Alabama,4903185,3814879,77.8
2,40,4,9,2,Alaska,731545,551562,75.4
3,40,4,8,4,Arizona,7278717,5638481,77.5
4,40,3,7,5,Arkansas,3017804,2317649,76.8


In [13]:
ny_pop = population_2019.loc[population_2019["NAME"] == "New York","POPESTIMATE2019"].values[0]

In [15]:
ny_confirmed/ny_pop

0.0030661738485822723

In [18]:
def get_unquarantine_change_rate():
    
    return 0
    